<a href="https://colab.research.google.com/github/gustavo-moretto/WebScrapping_ImoveisCuritiba/blob/main/imoveis_curitiba_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [91]:
def analise_imoveis( n_paginas, 
                    comprar_alugar=['aluguel', 'venda'], 
                    tipos_imoveis = ['apartamento_residencial', 'casa_residencial', 'condominio_residencial', 'cobertura_residencial', 'flat_residencial', 'kitnet_residencial', 'sobrado_residencial']):

  import requests
  import pandas as pd
  from bs4 import BeautifulSoup

  import seaborn as sns
  import matplotlib.pyplot as plt
  from time import sleep 

  conjunto_imoveis = {}
  c = 0
  for negocio in comprar_alugar:
    for pagina in range( n_paginas ):
        for tipo_imovel in tipos_imoveis:
          sleep(1)  
          # faz a requisição do site e verifica se esta foi bem-sucedida
          req = requests.get(f"https://www.vivareal.com.br/{negocio}/parana/curitiba/{tipo_imovel}/?pagina={pagina}")

          if req.status_code == 200:
              #print(f'Pag. {pagina}: Requisição bem sucedida!')
              content = req.content
          else:
            print(req.status.code)
            break
            
          # lê o arquivo html com a biblioteca BeautifulSoup
          soup = BeautifulSoup(content, 'html.parser')
            
          # seleciona os dados dos imóveis
          # para isso, encontramos todas as tags <a> com a "class" = "property-card__content-link js-card-title"
          imoveis = soup.find_all('a', attrs={'class':"property-card__content-link js-card-title"})
            
          for imovel in range(len(imoveis)):        
              c += 1
                
              indice_imovel = c
              # extrai a descrição do imóvel
              #descricao = imoveis[imovel].find("span", attrs={"class":"property-card__title js-cardLink js-card-title"}).text                 

              # extrai o endereço e o bairro do imóvel
              endereco = imoveis[imovel].find("span", attrs={"class":"property-card__address"}).text
              end_completo = endereco

              for elemento in range(len(endereco.split())):
                  if endereco.split()[elemento] == '-':
                      end = endereco.split()[elemento + 1]                  

                      # bairros com nomes compostos -> dessa forma, o nome eh extraido da maneira correta                 
                      if end in ['Santa', 'Sitio', 'Cristo', 'Vista', 'Boa', 'Água', 'Cidade', 'Campo', 'Capão']:
                        endereco = endereco.split()[elemento + 1] + ' ' + endereco.split()[elemento + 2]
                        # excluir a virgula do final do bairro
                        endereco = endereco[:-1]                    
                      elif end == 'Alto':
                        endereco = endereco.split()[elemento + 1] + ' ' + endereco.split()[elemento + 2] + ' ' + endereco.split()[elemento + 3]
                        # excluir a virgula do final do bairro
                        endereco = endereco[:-1]
                      
                      # caso nao haja o nome da rua, o bairro sera igual a 'PR'
                      # para extrair o bairro, devemos extrair o primeiro elemento da lista
                      elif end == 'PR':
                        endereco = endereco.split()[0][:-1]
                      else:
                        endereco = endereco.split()[elemento + 1]
                        # excluir a virgula do final do bairro
                        endereco = endereco[:-1]
                      break
              # extrai a área do imóvel (m²)
              # como existem imoveis com a area variavel, p. ex. 20-30, criei uma excecao
              # neste caso, a area do imovel sera igual a 0
              # creio que isso traz mais realidade ao dataset tambem
              try:
                area_m2 = float(imoveis[imovel].find("span", attrs={"class":"property-card__detail-value js-property-card-value property-card__detail-area js-property-card-detail-area"}).text)
              except ValueError:
                area_m2 = 0

              # extrai os itens do imóvel, como mobília, churrasqueira, academia...
              #try:
                  #itens = imoveis[imovel].find("ul", attrs={"class":"property-card__amenities"}).text.split()
              #except AttributeError:
                  #itens = []
              #try:
                  #if itens[-1] == '...':
                      #itens = itens[:-1]
                #except IndexError:
                    #pass

              # número de comodos (caso não possua vaga de garagem, por exemplo, será atribuído valor 0)
              comodos = []
              try:
                  for k in imoveis[imovel].find_all("span", attrs={"class":"property-card__detail-value js-property-card-value"}):
                      comodos.append(float(k.text))
              except ValueError:
                  comodos.append(0)
              if len(comodos) == 3:
                quartos = float(comodos[0])
                banheiros = float(comodos[1])
                vagas = float(comodos[2])
              else:
                quartos = 0
                banheiros = 0
                vagas = 0

              # extrai o valor do aluguel
              try: 
                preco = imoveis[imovel].find("p", attrs={'style':"display: block;"}).text                                                                                         
                try:          
                  if (negocio == 'aluguel' and len(preco.split()[1]) >= 5):            
                    preco = float(preco.split()[1])      
                    preco = preco * 1000                    
                  elif negocio == 'venda' and len(preco.split()) == 2:
                    preco = float(preco.split()[1])
                    preco = preco * 1000                    
                  elif negocio == 'venda' and len(preco.split()) == 6:
                    preco = float(preco.split()[1])
                    preco = preco * 1000                    
                  elif negocio == 'venda' and len(preco.split()) == 5:
                    preco = float(preco.split()[4])
                    preco = preco * 1000                    
                  else:
                    preco = float(preco.split()[1])                    
                    preco = preco * 1000                                                      
                except ValueError:
                  preco = imoveis[imovel].find("p", attrs={'style':"display: block;"}).text
                  preco = preco.replace('.', '')            
                  preco = float(preco.split()[1])
              except AttributeError:                                
                preco = 0                     

              # extrai o valor do condominio (se não existir, será 0)
              # no laco, caso o valor do condominio seja superior a 999.99 reais, sera multiplicado por 1000
              try:
                  condominio = imoveis[imovel].find("strong", attrs={'class':"js-condo-price"}).text
                  if len(condominio.split()[1]) == 3:
                    condominio = float(condominio.split()[1])
                  else:
                    condominio = float(condominio.split()[1])*1000             
              except AttributeError or ValueError:
                  condominio = 0

              conjunto_imoveis[c] = (negocio, tipo_imovel, indice_imovel, end_completo, endereco, area_m2, quartos, banheiros, vagas, preco, condominio)

  df = pd.DataFrame.from_dict(conjunto_imoveis)
  df = df.T
  df = df.rename(columns={ 0:'comprar_alugar', 1:'tipo_imovel', 2:'indice', 3:'end_completo', 4:'bairro', 5:'area_m2', 6:'quartos', 7:'banheiros', 8:'vagas_garagem', 9:'preco', 10:'condominio'})

  colunas_float = ['area_m2', 'quartos', 'banheiros', 'vagas_garagem', 'preco', 'condominio']
  for coluna in colunas_float: 
    df[coluna] = df[coluna].astype( float )
      
  print(f'Base de dados criada: {len(df)} imoveis cadastrados.')
  return df

In [132]:
class database_imoveis():

  def __init__(self, n_paginas):
    self.n_paginas = n_paginas

  def analise_imoveis( self, 
                    comprar_alugar=['aluguel', 'venda'], 
                    tipos_imoveis = ['apartamento_residencial', 'casa_residencial', 'condominio_residencial', 'cobertura_residencial', 'flat_residencial', 'kitnet_residencial', 'sobrado_residencial']):

    import requests
    import pandas as pd
    from time import sleep
    from bs4 import BeautifulSoup

    import seaborn as sns
    import matplotlib.pyplot as plt    

    from google.colab import files
    from datetime import date 

    conjunto_imoveis = {}
    c = 0
    for negocio in comprar_alugar:
      for pagina in range( self.n_paginas ):
          for tipo_imovel in tipos_imoveis:
            sleep(1)  
            # faz a requisição do site e verifica se esta foi bem-sucedida
            req = requests.get(f"https://www.vivareal.com.br/{negocio}/parana/curitiba/{tipo_imovel}/?pagina={pagina}")

            if req.status_code == 200:
                #print(f'Pag. {pagina}: Requisição bem sucedida!')
                content = req.content
            else:
              print(req.status.code)
              break
              
            # lê o arquivo html com a biblioteca BeautifulSoup
            soup = BeautifulSoup(content, 'html.parser')
              
            # seleciona os dados dos imóveis
            # para isso, encontramos todas as tags <a> com a "class" = "property-card__content-link js-card-title"
            imoveis = soup.find_all('a', attrs={'class':"property-card__content-link js-card-title"})
              
            for imovel in range(len(imoveis)):        
                c += 1
                  
                indice_imovel = c
                # extrai a descrição do imóvel
                #descricao = imoveis[imovel].find("span", attrs={"class":"property-card__title js-cardLink js-card-title"}).text                 

                # extrai o endereço e o bairro do imóvel
                endereco = imoveis[imovel].find("span", attrs={"class":"property-card__address"}).text
                end_completo = endereco

                for elemento in range(len(endereco.split())):
                    if endereco.split()[elemento] == '-':
                        end = endereco.split()[elemento + 1]                  

                        # bairros com nomes compostos -> dessa forma, o nome eh extraido da maneira correta                 
                        if end in ['Santa', 'Sitio', 'Cristo', 'Vista', 'Boa', 'Água', 'Cidade', 'Campo', 'Capão']:
                          endereco = endereco.split()[elemento + 1] + ' ' + endereco.split()[elemento + 2]
                          # excluir a virgula do final do bairro
                          endereco = endereco[:-1]                    
                        elif end == 'Alto':
                          endereco = endereco.split()[elemento + 1] + ' ' + endereco.split()[elemento + 2] + ' ' + endereco.split()[elemento + 3]
                          # excluir a virgula do final do bairro
                          endereco = endereco[:-1]
                        
                        # caso nao haja o nome da rua, o bairro sera igual a 'PR'
                        # para extrair o bairro, devemos extrair o primeiro elemento da lista
                        elif end == 'PR':
                          endereco = endereco.split()[0][:-1]
                        else:
                          endereco = endereco.split()[elemento + 1]
                          # excluir a virgula do final do bairro
                          endereco = endereco[:-1]
                        break
                # extrai a área do imóvel (m²)
                # como existem imoveis com a area variavel, p. ex. 20-30, criei uma excecao
                # neste caso, a area do imovel sera igual a 0
                # creio que isso traz mais realidade ao dataset tambem
                try:
                  area_m2 = float(imoveis[imovel].find("span", attrs={"class":"property-card__detail-value js-property-card-value property-card__detail-area js-property-card-detail-area"}).text)
                except ValueError:
                  area_m2 = 0

                # extrai os itens do imóvel, como mobília, churrasqueira, academia...
                #try:
                    #itens = imoveis[imovel].find("ul", attrs={"class":"property-card__amenities"}).text.split()
                #except AttributeError:
                    #itens = []
                #try:
                    #if itens[-1] == '...':
                        #itens = itens[:-1]
                  #except IndexError:
                      #pass

                # número de comodos (caso não possua vaga de garagem, por exemplo, será atribuído valor 0)
                comodos = []
                try:
                    for k in imoveis[imovel].find_all("span", attrs={"class":"property-card__detail-value js-property-card-value"}):
                        comodos.append(float(k.text))
                except ValueError:
                    comodos.append(0)
                if len(comodos) == 3:
                  quartos = float(comodos[0])
                  banheiros = float(comodos[1])
                  vagas = float(comodos[2])
                else:
                  quartos = 0
                  banheiros = 0
                  vagas = 0

                # extrai o valor do aluguel
                try: 
                  preco = imoveis[imovel].find("p", attrs={'style':"display: block;"}).text                                                                                         
                  try:          
                    if (negocio == 'aluguel' and len(preco.split()[1]) >= 5):            
                      preco = float(preco.split()[1])      
                      preco = preco * 1000                    
                    elif negocio == 'venda' and len(preco.split()) == 2:
                      preco = float(preco.split()[1])
                      preco = preco * 1000                    
                    elif negocio == 'venda' and len(preco.split()) == 6:
                      preco = float(preco.split()[1])
                      preco = preco * 1000                    
                    elif negocio == 'venda' and len(preco.split()) == 5:
                      preco = float(preco.split()[4])
                      preco = preco * 1000                    
                    else:
                      preco = float(preco.split()[1])                    
                      preco = preco * 1000                                                      
                  except ValueError:
                    preco = imoveis[imovel].find("p", attrs={'style':"display: block;"}).text
                    preco = preco.replace('.', '')            
                    preco = float(preco.split()[1])
                except AttributeError:                                
                  preco = 0                     

                # extrai o valor do condominio (se não existir, será 0)
                # no laco, caso o valor do condominio seja superior a 999.99 reais, sera multiplicado por 1000
                try:
                    condominio = imoveis[imovel].find("strong", attrs={'class':"js-condo-price"}).text
                    if len(condominio.split()[1]) == 3:
                      condominio = float(condominio.split()[1])
                    else:
                      condominio = float(condominio.split()[1])*1000             
                except AttributeError or ValueError:
                    condominio = 0

                conjunto_imoveis[c] = (negocio, tipo_imovel, indice_imovel, end_completo, endereco, area_m2, quartos, banheiros, vagas, preco, condominio)

    self.df = pd.DataFrame.from_dict(conjunto_imoveis)
    self.df = self.df.T
    self.df = self.df.rename(columns={ 0:'comprar_alugar', 1:'tipo_imovel', 2:'indice', 3:'end_completo', 4:'bairro', 5:'area_m2', 6:'quartos', 7:'banheiros', 8:'vagas_garagem', 9:'preco', 10:'condominio'})

    colunas_float = ['area_m2', 'quartos', 'banheiros', 'vagas_garagem', 'preco', 'condominio']
    for coluna in colunas_float: 
      self.df[coluna] = self.df[coluna].astype( float )
        
    print(f'Base de dados criada: {len(self.df)} imoveis cadastrados.')

    # download da base de dados
    download_basedados = str(input('Deseja fazer o download da base de dados [s/n]: '))
    if download_basedados.upper() == 'S':
      today = date.today()
      print(f'Download do arquivo df_imoveis_{today}.csv')
      self.df.to_csv(f'df_imoveis_{today}.csv')
      files.download(f'df_imoveis_{today}.csv')
    else:
      print('Base de dados nao foi baixada.')
      
    return self.df

In [138]:
database = database_imoveis(20)
df = database.analise_imoveis()
df.head()

KeyboardInterrupt: ignored

# Imports

In [5]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

# Analise Descritiva de Dados

In [134]:
df.describe(include='all')

,comprar_alugar,tipo_imovel,indice,end_completo,bairro,area_m2,quartos,banheiros,vagas_garagem,preco,condominio
count,491,491,491.0,491,491,491.000000,491.000000,491.000000,491.000000,4.910000e+02,491.000000
unique,2,7,491.0,427,75,NaN,NaN,NaN,NaN,NaN,NaN
top,venda,apartamento_residencial,1.0,"Centro, Curitiba - PR",Centro,NaN,NaN,NaN,NaN,NaN,NaN
freq,252,72,1.0,5,75,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,146.985743,2.535642,2.503055,1.851324,4.904174e+05,1469.148676
std,NaN,NaN,NaN,NaN,NaN,145.229522,1.331047,1.558875,1.837338,8.329734e+05,8764.836078
min,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000
25%,NaN,NaN,NaN,NaN,NaN,40.000000,1.000000,1.000000,1.000000,3.700000e+03,0.000000
50%,NaN,NaN,NaN,NaN,NaN,110.000000,3.000000,2.000000,2.000000,2.150000e+05,180.000000
75%,NaN,NaN,NaN,NaN,NaN,200.000000,3.000000,3.000000,2.000000,6.850000e+05,585.000000
